In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import RGBNirModule, RGBNirDataModule

dm = RGBNirDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
model = RGBNirModule.load_from_checkpoint('checkpoints/r18da-val_loss=6.45633-epoch=4.ckpt')
model.hparams

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb22747a310>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1268, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdown'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fb22747a310>
Traceback (most recent call last):
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/home/juan/miniconda3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1268, in _shutdown_workers
    if not self._shutdown:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_shutdo

"backbone":         resnet18
"datamodule":       {'batch_size': 512, 'num_workers': 10, 'pin_memory': True, 'train_trans': {'HorizontalFlip': {'p': 0.5}, 'VerticalFlip': {'p': 0.5}, 'Transpose': {'p': 0.5}, 'RandomRotate90': {'p': 0.5}}}
"early_stopping":   True
"optimizer":        Adam
"optimizer_params": {'lr': 0.001}
"pretrained":       True
"trainer":          {'gpus': 1, 'max_epochs': 30, 'logger': <pytorch_lightning.loggers.wandb.WandbLogger object at 0x7fb1df648100>, 'enable_checkpointing': True, 'overfit_batches': 0, 'deterministic': True, 'precision': 16, 'callbacks': [<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7fb1df648160>, <pytorch_lightning.callbacks.early_stopping.EarlyStopping object at 0x7fb1d449b910>, <pytorch_lightning.callbacks.progress.tqdm_progress.TQDMProgressBar object at 0x7fb1d449b940>, <pytorch_lightning.callbacks.model_summary.ModelSummary object at 0x7fb1d449b970>, <pytorch_lightning.callbacks.gradient_accumulation_scheduler.Gr

In [4]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for imgs, labels in tqdm(dl):
    preds = model.predict(imgs)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [02:02<00:00,  1.55s/it]


0.7288397723277075

In [9]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for imgs, observation_ids in tqdm(dl):
    preds = model.predict(imgs)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:24<00:00,  2.90it/s]


In [10]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
27184,21077774,5045 5273 5612 5095 389 5270 5332 5366 5053 59...
4228,10210359,393 663 833 3949 531 3149 821 716 414 910 76 1...
28768,21249204,8851 6568 5196 5226 5225 2648 869 7682 5936 71...
35228,21943515,5045 6547 2733 2386 5143 5366 2200 1743 5011 5...
34221,21827506,5045 3072 3034 5152 4999 5037 5024 3064 5025 5...


In [11]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
25717,20922092,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
30353,21416199,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4681,10221912,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
12817,10613746,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1340,10064142,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
5172,10244884,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20594,20363012,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
34633,21876321,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
20099,20307027,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
31958,21590367,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
